# How People Decide what they want to do
Directed graph approach

Generally people want to do a number of different things. For this I'm going to create a schema for this in a graph language that allows me to designate how much a `pop` desires to take a certain action. This will be used later when determining AI decisions. 

**Note** this notebook actualy builds the desires into the graph, overwriting existing ontology. 

In [3]:
import sys
import numpy as np
import pandas as pd
import altair as alt
sys.path.append('..')
import helpers.dbquery as db
import helpers.functions as f
import yaml, ssl, asyncio
import nb_black

ssl._create_default_https_context = ssl._create_unverified_context
asyncio.set_event_loop_policy(asyncio.WindowsSelectorEventLoopPolicy())
import nest_asyncio
# this is required for running in a Jupyter Notebook. 
nest_asyncio.apply()

In [4]:
res = db.run_query("g.V().hasLabel('pop').has('username','userbill').valueMap()")
pops = [db.clean_node(n) for n in res]
pops[0]

{'conformity': '0.197',
 'literacy': '0.822',
 'aggression': '0.384',
 'constitution': '0.615',
 'objid': '5771833830958',
 'faction_no': '2',
 'name': 'Heznius Ying',
 'isInFaction': '7100167726406',
 'industry': '0.4995',
 'wealth': '0.66075',
 'faction_loyalty': '0.49550000000000005',
 'username': 'userbill',
 'objtype': 'pop',
 'id': '5771833830958'}

Each population wants to do everything to a degree, the amount of desire to do that thing is expressed by the edge weight. 
* Attack a population
* Focus on improving literacy
* Focus on improving industry

# Desires as Objects

## Desire with targets.
Both factions and pops can have desire. Action is guided by desire based on the `max(desire.weight)`.
`desire` is an edge, the type of that desire is a property of that edge, and the edge weight is the amount of desire. The target (`node2`) is the recipient. 

Examples:
* faction wants trade with faction
* pop wants war with another pop
* pop wants faction to go to war with faction

## Desire without targets.

Desires without targets must link to an objective. That objective can be it's own node.


In [5]:
# # Drop the items, if they exist. 
# db.run_query("g.V().hasLabel('objective').has('username','notebook').drop()")
# objectives_yaml = yaml.safe_load(open("desires.yaml"))['objectives']
# data = {"nodes":objectives_yaml,'edges':[]}
# # Then Create the nodes and add them to the DB
# db.upload_data(data,verbose=False)
# After creating the nodes, pulling them into the notebook for reference
res = db.run_query("g.V().hasLabel('objective').valueMap()")
objectives = [db.clean_node(n) for n in res]
# objectives


# population wants to improve industry

populations want to improve industry when:
* they are not wealty
* they are at war




In [6]:
# Marginal return on 
n = 2
ind_df = pd.DataFrame(np.sort([float(p['wealth']) for p in pops]),columns=['wealth'])
ind_df['base'] = range(len(ind_df))
ind_df['desires_industry'] = ind_df['wealth'].apply(lambda x: ((x+1)**(1-n) - 1)/(1-n))
ind_df['desire_base'] = ind_df['base'].apply(lambda x: ((x+1)**(1-n) - 1)/(1-n))
alt.Chart(ind_df).mark_line().encode(x='base',y='desire_base').properties(title="Desire relative to the base attribute")

alt.Chart(...)

In [7]:
alt.Chart(ind_df).mark_line().encode(x='wealth:N',y='desires_industry').properties(title="Desires wealth industry relative to industry")

alt.Chart(...)

## feeding that desire to the populations


In [8]:
objectives[0],pops[0]

({'type': 'industry',
  'weight': '0.5',
  'leadingAttribute': 'wealth',
  'comment': 'build factories, skyscrapers, infrastructure',
  'username': 'notebook',
  'objtype': 'objective',
  'objid': '0321508144812',
  'id': '0321508144812'},
 {'conformity': '0.197',
  'literacy': '0.822',
  'aggression': '0.384',
  'constitution': '0.615',
  'objid': '5771833830958',
  'faction_no': '2',
  'name': 'Heznius Ying',
  'isInFaction': '7100167726406',
  'industry': '0.4995',
  'wealth': '0.66075',
  'faction_loyalty': '0.49550000000000005',
  'username': 'userbill',
  'objtype': 'pop',
  'id': '5771833830958'})

In [9]:
def get_desire(x):
       return np.round(((float(x)+1)**(1-n) - 1)/(1-n),3)

edges = []
for p in pops:
    for o in objectives:
        edge = {'label':'desires',
                'node1':p['objid'],
                'node2':o['objid'],
                'weight':get_desire(p[o['leadingAttribute']])}
        edges.append(edge)

In [10]:
pd.DataFrame(edges)

,label,node1,node2,weight
0,desires,5771833830958,0321508144812,0.398
1,desires,5771833830958,9021333780224,0.333
2,desires,5771833830958,8268144304568,0.277
3,desires,5771833830958,4248820160134,0.451
4,desires,5771833830958,8681411407190,0.451
5,desires,4244612303621,0321508144812,0.369
6,desires,4244612303621,9021333780224,0.368
7,desires,4244612303621,8268144304568,0.338
8,desires,4244612303621,4248820160134,0.369
9,desires,4244612303621,8681411407190,0.369


In [14]:
db.create_edge(edges[0])

"g.V().has('objid','5771833830958').addE('desires').property('username','notebook').property('weight','0.398').to(g.V().has('objid','0321508144812'))"

In [15]:
db.upload_data({'nodes':[],'edges':edges},verbose=False)

g.V().has('objid','5771833830958').addE('desires').property('username','notebook').property('weight','0.398').to(g.V().has('objid','0321508144812'))
g.V().has('objid','5771833830958').addE('desires').property('username','notebook').property('weight','0.333').to(g.V().has('objid','9021333780224'))
g.V().has('objid','5771833830958').addE('desires').property('username','notebook').property('weight','0.277').to(g.V().has('objid','8268144304568'))
g.V().has('objid','5771833830958').addE('desires').property('username','notebook').property('weight','0.451').to(g.V().has('objid','4248820160134'))
g.V().has('objid','5771833830958').addE('desires').property('username','notebook').property('weight','0.451').to(g.V().has('objid','8681411407190'))
g.V().has('objid','4244612303621').addE('desires').property('username','notebook').property('weight','0.369').to(g.V().has('objid','0321508144812'))
g.V().has('objid','4244612303621').addE('desires').property('username','notebook').property('weight','0.36

exception calling callback for <Future at 0x1b250f6e940 state=finished returned NoneType>
Traceback (most recent call last):
  File "C:\Users\william.harding\anaconda3\envs\exoplanets\lib\concurrent\futures\_base.py", line 324, in _invoke_callbacks
    callback(self)
  File "C:\Users\william.harding\anaconda3\envs\exoplanets\lib\site-packages\gremlin_python\driver\connection.py", line 72, in cb
    done = self._executor.submit(self._receive)
  File "C:\Users\william.harding\anaconda3\envs\exoplanets\lib\concurrent\futures\thread.py", line 151, in submit
    raise RuntimeError('cannot schedule new futures after shutdown')
RuntimeError: cannot schedule new futures after shutdown
Unclosed client session
client_session: <aiohttp.client.ClientSession object at 0x000001B250EE5860>


g.V().has('objid','3103900522647').addE('desires').property('username','notebook').property('weight','0.291').to(g.V().has('objid','8268144304568'))
g.V().has('objid','3103900522647').addE('desires').property('username','notebook').property('weight','0.411').to(g.V().has('objid','4248820160134'))
g.V().has('objid','3103900522647').addE('desires').property('username','notebook').property('weight','0.411').to(g.V().has('objid','8681411407190'))
g.V().has('objid','7482929105840').addE('desires').property('username','notebook').property('weight','0.353').to(g.V().has('objid','0321508144812'))
g.V().has('objid','7482929105840').addE('desires').property('username','notebook').property('weight','0.337').to(g.V().has('objid','9021333780224'))
g.V().has('objid','7482929105840').addE('desires').property('username','notebook').property('weight','0.258').to(g.V().has('objid','8268144304568'))
g.V().has('objid','7482929105840').addE('desires').property('username','notebook').property('weight','0.36